In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

import re

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time


from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options


In [22]:
def geTopographic(topographicUrl):
    
    # receber o nome da cidade 
    # pega os dados de localização e altitude média  minimas e máximas
    # retorna uma lista com os dados
    
   
    print(topographicUrl)
    r2 = requests.get(topographicUrl)
    soup2 = BeautifulSoup(r2.text, 'html.parser')
    
    # Verificação de erro caso a cidade não exista no site 
    if r2.status_code == 404 or soup2 == None:
        return 'Cidade não encontrada'
    
    else:

        # pegar todos os itens que posuuem <div>

        info = soup2.find_all('div')
        text = [i.text for i in info]

        #pegar partes do tetxo que iniciam por 'Nome:'

        data = [t for t in text if t.startswith('Nome:')]
        
        """
            #pegar os valores de localização e altitude média  minimas e máximas
            data = data[-1].split(',')[-1]
            #deletar string '\xa0m'
            data = data.replace('\xa0m', '')
            #separa ao encontrar letras maiusculas
            data = re.findall('[A-Z][^A-Z]*', data)
            
        """
        
        if data == []:
            return [None, None, None, None]
        
        else:
        
            data = data[-1].split(',')[-1]
            data  = data.replace('\xa0m', '')


            #substitui Regex por split
            l = data.split(')')[0].split('(')[-1]
            print("L_>",l)
            aMed = data.split(')')[-1].split(':')[1].split('Altitude mínima')[0]
            aMax= data.split(')')[-1].split(':')[-1].split('Altitude mínima')[0]
            aMin = data.split(')')[-1].split(':')[2].split('Altitude máxima')[0]
            
            data = [l, aMed, aMax, aMin]
            
            return data



In [23]:
def GetKoppenClassifyPt():
    url = r'https://pt.wikipedia.org/wiki/Classifica%C3%A7%C3%A3o_clim%C3%A1tica_de_K%C3%B6ppen-Geiger'

    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')

    # get class 'mw-headline'
    headline = soup.find_all('span', class_='mw-headline')
    hlist = [h.text for h in headline]
    hlist
    # deixar apenas as classes que iniciam com Climas
    hlist = [h for h in hlist if h.startswith('Clima')]

    # capturar elemento <\li> dos elementos de classe 'mw-headline' em hlist que estiverem com um ::marker
    # get class 'mw-parser-output'

    clines = soup.find_all('li')
    #filtrar apenas os elementos que terminam com )
    citilist = [c.text for c in clines if c.text.endswith(')')]
    #deletar os ultimos 5 elementos da lista
    citilist = citilist[:-5]
    climate_dict = {} # chaves = hlist, valores = citilist

    for h in hlist:
        climate_dict[h] = [] #cria uma lista vazia para cada chave

    for c in citilist:
        cityCode = c[-4::].strip('()') # codigo da cidade
        for h in hlist:
            hCode = h.split('−')[1] # codigo do clima
            if cityCode in hCode: # se o codigo da cidade estiver no codigo do clima
                climate_dict[h].append(c)
                
    # criar dataframe com uma coluna para cidade e uma para o clima da respectiva cidade
    df = pd.DataFrame(columns=['Cidade', 'Clima'])



    for k, v in climate_dict.items():
        temp = pd.DataFrame({'Cidade': v, 'Clima': k})
        df =  pd.concat([df, temp], ignore_index=True)

    # retirar o codigo da cidade do nome da cidade e adicionar a coluna cidade

    df['Cidade'] =  df['Cidade'].apply(lambda x: x.split('(')[0])

In [24]:
def GetKoppenClassifyEn():
    
    #Retornar um dataframe com as cidades e seus respectivos climas de acordo com a classificação de Köppen
    
    
    r = requests.get('https://en.wikipedia.org/wiki/K%C3%B6ppen_climate_classification')
    soup = BeautifulSoup(r.text, 'html.parser')

    # get class 'mw-headline'
    Headline = soup.find_all('span', {'class': 'mw-headline'})
    Hlist = [h.text for h in Headline]

    Hlist = [h for h in Hlist if 'climat' in h[-8:-1]]
    #eliminar elemnetos que iniciam com group
    Hlist = [h for h in Hlist if 'Group' not in h[0:5]]
    #retirrar 'Marine west coast climate',
    Hlist.remove('Marine west coast climate')

    #capturar elemento <\li> dos elementos de classe 'mw-headline' em hlist que estiverem com um ::marker
    # get class 'mw-parser-output'

    Clines = soup.find_all('li')
    #filtrar apenas os elementos que terminam com )
    Citilist = [c.text for c in Clines if c.text.endswith(')')]
    #deletar os ultimos 15 elementos da lista
    Citilist = Citilist[:-21]
    Climate_dict = {} # chaves = hlist, valores = citilist
    codeClimate = [] # lista com os codigos do clima

    for h in Hlist:
        Climate_dict[h] = [] #cria uma lista vazia para cada chave

    for c in Citilist:
        separated = c.split('(')[1].split(')')[0].split(', ')
        codeClimate.append(separated[0]) # lista com os codigos do clima
    
        
        for h in Hlist:
            HCode = h.split(":")[0] # codigo do clima
            
            if HCode == separated[0]:
                Climate_dict[h].append(c) # adiciona o nome da cidade na lista do clima correspondente
                
    #criar um dataframe com os dados

    # criar dataframe com uma coluna para cidade e uma para o clima da respectiva cidade
    df = pd.DataFrame(columns=['Cidade', 'Clima'])



    for k, v in Climate_dict.items():
        temp = pd.DataFrame({'Cidade': v, 'Clima': k})
        df =  pd.concat([df, temp], ignore_index=True)

    # retirar o codigo da cidade do nome da cidade 

    df['Cidade'] = df['Cidade'].apply(lambda x: x.split('(')[0])
    
    return df

In [25]:
# # Adicionar colunas de localização e altitude média  minimas e máximas

# df['Localização'] = df['Cidade'].apply(lambda x: geTopographic(x)[0])
# df['Altitude Média'] = df['Cidade'].apply(lambda x: geTopographic(x)[1])
# df['Altitude Mínima'] = df['Cidade'].apply(lambda x: geTopographic(x)[2])
# df['Altitude Máxima'] = df['Cidade'].apply(lambda x: geTopographic(x)[3])


In [26]:
def Browser(city):


        
    # Configuração do ChromeDriver em modo headless
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-gpu')
    # Configuração do ChromeDriver
    chrome_path = r'C:\Users\Usuário\Downloads\chromedriver-win32\chromedriver.exe'   # Substitua pelo caminho do seu ChromeDriver
    driver = webdriver.Chrome(executable_path=chrome_path, options=chrome_options)

    # Abre o site
    driver.get("https://pt-br.topographic-map.com/");

    # Espera explicitamente até que a caixa de pesquisa esteja presente
    try:
        search_box = WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.XPATH, '//form[@title="Pesquisar um mapa topográfico"]/input[@name="query"]'))
        )

        # Limpar qualquer texto existente na caixa de pesquisa
        search_box.clear()

        # Enviar a consulta "Beirute"
        search_box.send_keys(city)

        # Enviar a tecla Enter
        search_box.send_keys(Keys.RETURN)

        # Aguarde alguns segundos para os resultados serem exibidos
        time.sleep(1)
        #fazer scrape dos dados
        

    

    except Exception as e:
        print(f"Erro: {e}")

    finally:
      
       
        return geTopographic(driver.current_url)


In [27]:
def GetTable(url):
    #MAIS RAPIDO
    # Configuração do ChromeDriver em modo headless
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-gpu')

    # Configuração do ChromeDriver
    chrome_path = r'C:\Users\Usuário\Downloads\chromedriver-win32\chromedriver.exe'  # Substitua pelo caminho do seu ChromeDriver
    driver = webdriver.Chrome(executable_path=chrome_path, options=chrome_options)

    driver.get(url);

    # Obtenha o HTML da página após a execução de scripts
    html = driver.page_source

    # Use o BeautifulSoup para analisar o HTML
    soup = BeautifulSoup(html, 'html.parser')

    # Armazene referências aos elementos fora do loop
    city_element = soup.find('div', {'class': 'col-10 m_city_name fs0'})
    country_element = soup.find('div', {'class': 'col-12 city_country fs0'})
    table = soup.find('table', {'class': 'climateTable'})

    # Verifique se os elementos foram encontrados antes de acessá-los
    city = city_element.text if city_element else None
    country = country_element.text if country_element else None
    print(city, country)

    if table:
        rows = table.find_all('tr')[1:]

        # Criar listas para armazenar os dados
        months, min_temps, max_temps, rainfalls, rain_days = [], [], [], [], []

        # Iterar sobre as linhas e extrair os dados
        for row in rows:
            columns = row.find_all('td')
            
            if len(columns) < 5:
                continue
            months.append(columns[0].text.strip())
            min_temps.append(float(columns[1].text.strip()))
            max_temps.append(float(columns[2].text.strip()))
            rainfalls.append(float(columns[3].text.strip()))
            rain_days.append(float(columns[4].text.strip()))

        # Feche o navegador após a extração de dados
        #driver.quit()

        return city, min_temps, max_temps, rainfalls, rain_days

    else:
        # Se a tabela não foi encontrada, feche o navegador e retorne None
        #driver.quit()
        return None, None, None, None, None


In [28]:

def GetTable2(url):
    
    #MAIS RAPIDO
    # Configuração do ChromeDriver em modo headless
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-gpu')

    # Configuração do ChromeDriver
    chrome_path = r'C:\Users\Usuário\Downloads\chromedriver-win32\chromedriver.exe'  # Substitua pelo caminho do seu ChromeDriver
    driver = webdriver.Chrome(executable_path=chrome_path, options=chrome_options)
  
    driver.get(url)

    # Obtenha o HTML da página após a execução de scripts
    html = driver.page_source

    # Use o BeautifulSoup para analisar o HTML
    soup = BeautifulSoup(html, 'html.parser')
    
    
  
    
    
    
    # Armazene referências aos elementos fora do loop
    city_element = soup.find('div', {'class': 'col-10 m_city_name fs0'})
    country_element = soup.find('div', {'class': 'col-12 city_country fs0'})
    table = soup.find('table', {'class': 'climateTable'})
    if table is None:
        print('table is none')
        return None, None, None, None, None
    
    
    rows = table.find_all('tr')[1:]

    # Obtenha o texto dos elementos fora do loop
    city = city_element.text if city_element else None
    country = country_element.text if country_element else None
    print(city,country)
    # Criar listas para armazenar os dados
    months, min_temps, max_temps, rainfalls, rain_days = [], [], [], [], []

    # Iterar sobre as linhas e extrair os dados
    for row in rows:
        columns = row.find_all('td')
        
        if len(columns)!=5:
            print("Erro")
            break
        
    
        months.append(columns[0].text.strip())
        min_temps.append(float(columns[1].text.strip()))
        max_temps.append(float(columns[2].text.strip()))
        rainfalls.append(float(columns[3].text.strip()))
        rain_days.append(float(columns[4].text.strip()))
        

    # Feche o navegador após a extração de dados
    #driver.quit()

    return city, min_temps, max_temps, rainfalls, rain_days
    

In [29]:
koppenData = GetKoppenClassifyEn()

In [30]:
# #Adicionar ao df as colunas de localização e altitude média, mínima e máxima para cada cidade usando a função Browser

# localizaçaõ = []
# AltitudeMédia = []
# AltitudeMínima = []
# AltitudeMáxima = []

# for c in df['Cidade']:
    
#     print(c)
#     response = Browser(c)
#     if response != 'Cidade não encontrada' and len(response) == 4:
        
#         localizaçaõ.append(response[0])
#         AltitudeMédia.append(response[1])
#         AltitudeMáxima.append(response[2])
#         AltitudeMínima.append(response[3])
        
        
#     else:
        
#         localizaçaõ.append(None)   
#         AltitudeMédia.append(None) 
#         AltitudeMínima.append(None)
#         AltitudeMáxima.append(None)
     
# df['Localização'] = localizaçaõ
# df['Altitude Média'] = AltitudeMédia
# df['Altitude Mínima'] = AltitudeMínima
# df['Altitude Máxima'] = AltitudeMáxima

# #descobrir qual cidade da erro e também diferença entre df['Cidade'] e CityList;

In [31]:
# # MAIN

# #Adicionar ao df as colunas de localização e altitude média, mínima e máxima para cada cidade usando a função Browser

# localizaçaõ = []
# AltitudeMédia = []
# AltitudeMínima = []
# AltitudeMáxima = []


# mediaMinTemp = []
# mediaMaxTemp = []
# Precipitação = []
# QtDiasChuva = []

# df  = pd.DataFrame()
# DicTable = {}

# #1 - 3519 --> range(1,3520)
 
# for i in range(17,21):
#     print(i)
#     r = GetTable2(f'https://worldweather.wmo.int/en/city.html?cityId={i}')
#     print(r[0])
#     DicTable[r[0]] = r[1:]



# df['Cidade'] = DicTable.keys()
# df['Cidade'] = df['Cidade'].str.replace('✚','')



# for c in DicTable.keys():
    
    
#     if c != None:
        
#         print("VAlor entrada:  ",c)
#         c = c.replace('✚','')
        
#         response = Browser(c)
#         if response != 'Cidade não encontrada' and len(response) == 4:
            
#             localizaçaõ.append(response[0])
#             AltitudeMédia.append(response[1])
#             AltitudeMáxima.append(response[2])
#             AltitudeMínima.append(response[3])
            
            
#             r = GetTable2(f'https://worldweather.wmo.int/en/city.html?cityId={i}')
#             print('Valor Request',r[0])
            
#             if str(r[0]).replace('✚','') == c and mediaMaxTemp != None:
                
#                 mediaMinTemp.append(np.mean(r[1]))
#                 mediaMaxTemp.append(np.mean(r[2]))
#                 Precipitação.append(np.mean(r[3]))
#                 QtDiasChuva.append(np.mean(r[4]))
                
#             else:
                
#                 print("interno")
#                 mediaMaxTemp.append(None)
#                 mediaMinTemp.append(None)
#                 Precipitação.append(None)
#                 QtDiasChuva.append(None)
                
#         else:
            
#             print("Medio")
#             localizaçaõ.append(None)   
#             AltitudeMédia.append(None) 
#             AltitudeMínima.append(None)
#             AltitudeMáxima.append(None)
            
#             mediaMaxTemp.append(None)
#             mediaMinTemp.append(None)
#             Precipitação.append(None)
#             QtDiasChuva.append(None)
#     else:

#             print("Externo")
#             localizaçaõ.append(None)   
#             AltitudeMédia.append(None) 
#             AltitudeMínima.append(None)
#             AltitudeMáxima.append(None)
            
#             mediaMaxTemp.append(None)
#             mediaMinTemp.append(None)
#             Precipitação.append(None)
#             QtDiasChuva.append(None)
                
# df['Localização'] = localizaçaõ
# df['Altitude Média'] = AltitudeMédia
# df['Altitude Mínima'] = AltitudeMínima
# df['Altitude Máxima'] = AltitudeMáxima
# df['Média Mínima Temperatura'] = mediaMinTemp
# df['Média Máxima Temperatura'] = mediaMaxTemp
# df['Precipitação'] = Precipitação
# df['Qt Dias Chuva'] = QtDiasChuva     

In [32]:
# df

In [33]:
koppenData[0:10]['Cidade']

0                                Apia, Samoa 
1                      Balikpapan, Indonesia 
2                         Davao, Philippines 
3                       Easter Island, Chile 
4    Fort Lauderdale, Florida, United States 
5                         Georgetown, Guyana 
6                Hilo, Hawaii, United States 
7                   Honiara, Solomon Islands 
8           Innisfail, Queensland, Australia 
9                            Ishigaki, Japan 
Name: Cidade, dtype: object

In [34]:
#Para cada cidade de koppenData["Cidade"] pegar os dados usando da função Browser e adicionar ao df
y = []
for cd in koppenData[0:10]['Cidade']:
    y.append(Browser(cd))
    


C:\Users\Usuário\AppData\Local\Temp\ipykernel_20820\851715862.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_path, options=chrome_options)


https://pt-br.topographic-map.com/map-4lm1gp/Apia/
L_> -13.99446 -171.92491 -13.67446 -171.60491
https://pt-br.topographic-map.com/map-cvkh3q/Balikpapan/
L_> -1.28233 116.72880 -1.01330 117.02267
https://pt-br.topographic-map.com/map-vxln51/Davao-City/
L_> 6.95621 125.21759 7.58647 125.74485
https://pt-br.topographic-map.com/map-f8j1kl/Ilha-de-P%C3%A1scoa/
L_> -27.40200 -109.67958 -26.85500 -109.00299
https://pt-br.topographic-map.com/map-ss4b18/Fort-Lauderdale/
L_> 26.07961 -80.20364 26.21192 -80.08715
https://pt-br.topographic-map.com/map-h3qdgt/City-of-Georgetown/
L_> 6.76709 -58.17472 6.82755 -58.10527
https://pt-br.topographic-map.com/map-hg3ntp/Hilo-CDP/
L_> 19.62100 -155.18441 19.74913 -154.99044
https://pt-br.topographic-map.com/
https://pt-br.topographic-map.com/map-6xp5tf/Innisfail/
L_> -17.56417 145.99114 -17.48417 146.07114
https://pt-br.topographic-map.com/


In [35]:
y

[['-13.99446 -171.92491 -13.67446 -171.60491', ' 174', ' 1.148', ' 0'],
 ['-1.28233 116.72880 -1.01330 117.02267', ' 21', ' 129', ' -5'],
 ['6.95621 125.21759 7.58647 125.74485', ' 235', ' 2.627', ' -1'],
 ['-27.40200 -109.67958 -26.85500 -109.00299', ' 5', ' 501', ' 0'],
 ['26.07961 -80.20364 26.21192 -80.08715', ' 5', ' 38', ' -4'],
 ['6.76709 -58.17472 6.82755 -58.10527', ' 2', ' 11', ' -3'],
 ['19.62100 -155.18441 19.74913 -154.99044', ' 193', ' 728', ' 0'],
 [None, None, None, None],
 ['-17.56417 145.99114 -17.48417 146.07114', ' 12', ' 233', ' -2'],
 [None, None, None, None]]

In [36]:
koppenData

,Cidade,Clima
0,"Apia, Samoa",Af: Tropical rainforest climate
1,"Balikpapan, Indonesia",Af: Tropical rainforest climate
2,"Davao, Philippines",Af: Tropical rainforest climate
3,"Easter Island, Chile",Af: Tropical rainforest climate
4,"Fort Lauderdale, Florida, United States",Af: Tropical rainforest climate
...,...,...
370,"Scott Base, Antarctica",EF: Ice cap climate
371,"Showa Station, Antarctica",EF: Ice cap climate
372,"Summit Camp, Greenland, Denmark",EF: Ice cap climate
373,"Ushakov Island, Russia",EF: Ice cap climate
